In [7]:
from nltk import sent_tokenize
import nltk
import string
import numpy as np

In [8]:
filename = 'Klingon_Train.txt'
file = open(filename, encoding="utf8")
text = file.read()

f = open("README.txt", "w")

In [9]:
#Part 1
words = text.split()

#Counts how many times a tag appears in the file.
dictionary = {}
dictionary['<s>'] = 1

#Counts the number of times each "word" appears as a specific tag.
word_counts = {}

#Create a string of tags only.
tagOnly = "<s> "

#Create a string of words only.
wordOnly = ""

f.write("Word Probabilities To Tags \n")
#Collect counts
for word in words:
    newWord = word.split('/')
    tagOnly = tagOnly + newWord[1] + " "
    wordOnly = wordOnly + newWord[0] + " "
    if word not in word_counts:
        word_counts[word] = 1
    else:
        word_counts[word] = word_counts[word] + 1
        
    if newWord[1] not in dictionary:
        dictionary[newWord[1]] = 1
    else:
        dictionary[newWord[1]] = dictionary[newWord[1]] + 1
        
for word in word_counts:
    f.write("\n")
    newWord = word.split('/')
    for tag in dictionary:
        if (newWord[0] + "/" + tag) != word:
                f.write(tag + " : " + newWord[0] + " --> " + str(0.1) + '\n')
        else:
                f.write(tag + " : " + newWord[0] + " --> " + str(word_counts[word]/dictionary[newWord[1]]+0.1) + '\n')
                

In [10]:
#Part 2

#Create counts of bigrams
bigram_dict = {}

f.write("\n \n Tag Probability Given Previous Tag \n")

theTags = tagOnly.split()
for i in range(len(theTags) - 1):
    if theTags[i] not in bigram_dict:
        bigram_dict[theTags[i]] = {}
        bigram_dict[theTags[i]][theTags[i+1]] = 1
    else: 
        if theTags[i+1] not in bigram_dict[theTags[i]]:
            bigram_dict[theTags[i]][theTags[i+1]] = 1
        else:
            bigram_dict[theTags[i]][theTags[i+1]] = bigram_dict[theTags[i]][theTags[i+1]] + 1

            
for entry in bigram_dict:
    f.write(entry.upper() + " Transition Probabilities \n")
    f.write("________________________________________" + "\n")
    for tag in dictionary:
        if tag not in bigram_dict[entry]:
                f.write(entry + " " + tag + ": " + str(0.1) + "\n")
    for subentry in bigram_dict[entry]:
        f.write(entry + " " + subentry + ": " + str((bigram_dict[entry][subentry]/dictionary[entry]) + 0.1) + "\n")
        


In [11]:
#Part 3

sentence = wordOnly.split()

msg = "tera’ngan legh yaS".split()

keys = ['N', 'V', 'PRO', 'CONJ']

T = len(keys)

W = len(msg)

Score = np.zeros((T, W))

Backptr = np.zeros((T, W))

finalString = ""

#Initialization Step
for t in range(0, T):
    finalString = msg[0] + '/' + keys[t]
    if msg[0] not in wordOnly.split():
        if keys[t] not in bigram_dict['<s>']:
            Score[t][0] = ((0.1/(1))+0.1) * ((0/1 + 0.1))
    else:
        if finalString not in word_counts:
            if keys[t] not in bigram_dict['<s>']:
                Score[t][0] = ((0/(1))+0.1) * ((0/1) + 0.1)
            else:
                Score[t][0] = ((0/(1))+0.1) * ((bigram_dict['<s>'][keys[t]]/1) + 0.1)
        else:
            Score[t][0] = ((word_counts[finalString]/(1))+0.1) * (((bigram_dict['<s>'][keys[t]]/1)) + 0.1)
    
    Backptr[t][0] = 0
    
#Dynamic Step        
for w in range(1, W):
    for t in range(0, T):
        finalString = msg[w] + '/' + keys[t]
        maxScore = 0
        for j in range(0, T):
            if keys[j] in bigram_dict[keys[t]]:
                if Score[j][w-1] * bigram_dict[keys[t]][keys[j]] > maxScore:
                    maxScore = Score[j][w-1] * bigram_dict[keys[t]][keys[j]]
                    Backptr[t][w] = j #i am doing index 0 so adding 1. In this case 1 = 0 in our array.
            else:
                if Score[j][w-1] * 0.1 > maxScore:
                    maxScore = Score[j][w-1] * 0.1
                    Backptr[t][w] = j  #i am doing index 0 so adding 1. In this case 1 = 0 in our array.
        
        #Since we know the last 2 words are just not in our training at all, We can simply address case where the word is not 
        #present inour training corpus and leave the implementation of other cases until we wish to expand on this.
        if msg[w] not in wordOnly.split():
            Score[t][w] = ((0.1/dictionary[keys[t]])+0.1) * maxScore

#Backtracking step.           
Sequence = [0, 0, 0]

w = W - 2

while w >= 0:
    Sequence[w] = int(Backptr[Sequence[w+1]][w+1])
    w = w - 1

f.write("\n")
f.write("The likely tags for the sentence seem to be " + msg[0] + "/" + keys[int(Sequence[0])] + " " + msg[1] + "/" + keys[int(Sequence[1])] + " " + msg[2] + "/" + keys[int(Sequence[2])])
f.write("\n The message definitely starts with 'Human', but since we do not know what legh and yaS mean, we can only speculate. The subject seems to be yaS.")
    

146

In [12]:
f.close()